In [26]:
#from google.colab import drive

import os
import sys
import re
import string
import nltk
import pandas as pd
import numpy as np
import keras

from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dropout, Dense, Bidirectional,  Flatten, Input, GRU
from keras.layers.embeddings import Embedding

from unidecode import unidecode

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

from gensim.models import KeyedVectors
from gensim.models import Word2Vec

nltk.download('stopwords')
stopwords=stopwords.words('english')

T = TweetTokenizer()
#drive.mount('/content/drive')

#!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
EMBEDDING_FILE = '../GoogleNews-vectors-negative300.bin'

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/beuvry_j/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def value_sentiment(x):
  if x == "neutral":
    return 0
  if x == "positive":
    return 1
  if x == "negative":
    return -1

In [3]:
def onehot (tweets,length):
  onh = []
  for tweet in tweets:
      onh_seq = np.zeros(length) 
      for i in tweet:
        onh_seq[i -1] = 1
      onh.append(onh_seq)
  return onh

In [4]:
#replace emojy
emojis = {
    u":‑\)":"☺️",
    u":\)":"☺️",
    u":-\]":"☺️",
    u":\]":"☺️",
    u":-3":"☺️",
    u":3":"☺️",
    u":->":"☺️",
    u":>":"☺️",
    u"8-\)":"☺️",
    u":o\)":"☺️",
    u":-\}":"☺️",
    u":\}":"☺️",
    u":-\)":"☺️",
    u":c\)":"☺️",
    u":\^\)":"☺️",
    u"=\]":"☺️",
    u"=\)":"☺️",
    u":‑D":"😃",
    u":D":"😃",
    u"8‑D":"😃",
    u"8D":"😃",
    u"X‑D":"😃",
    u"XD":"😃",
    u"=D":"😃",
    u"=3":"😃",
    u"B\^D":"😃",
    u":-\)\)":"😃",
    u":‑\(":"☹️",
    u":-\(":"☹️",
    u":\(":"☹️",
    u":‑c":"☹️",
    u":c":"☹️",
    u":‑<":"☹️",
    u":<":"☹️",
    u":‑\[":"☹️",
    u":\[":"☹️",
    u":-\|\|":"☹️",
    u">:\[":"☹️",
    u":\{":"☹️",
    u":@":"☹️",
    u">:\(":"☹️",
    u":'‑\(":"😭",
    u":'\(":"😭",
    u":'‑\)":"😃",
    u":'\)":"😃",
    u"D‑':":"😨",
    u"D:<":"😨",
    u"D:":"😧",
    u"D8":"😧",
    u"D;":"😧",
    u"D=":"😧",
    u"DX":"😧",
    u":‑O":"😮",
    u":O":"😮",
    u":‑o":"😮",
    u":o":"😮",
    u":-0":"😮",
    u"8‑0":"😮",
    u">:O":"😮",
    u":-\*":"😗",
    u":\*":"😗",
    u":X":"😗",
    u";‑\)":"😉",
    u";\)":"😉",
    u"\*-\)":"😉",
    u"\*\)":"😉",
    u";‑\]":"😉",
    u";\]":"😉",
    u";\^\)":"😉",
    u":‑,":"😉",
    u";D":"😉",
    u":‑P":"😛",
    u":P":"😛",
    u"X‑P":"😛",
    u"XP":"😛",
    u":‑Þ":"😛",
    u":Þ":"😛",
    u":b":"😛",
    u"d:":"😛",
    u"=p":"😛",
    u">:P":"😛",
    u":‑/":"😕",
    u":/":"😕",
    u":-[.]":"😕",
    u">:[(\\\)]":"😕",
    u">:/":"😕",
    u":[(\\\)]":"😕",
    u"=/":"😕",
    u"=[(\\\)]":"😕",
    u":L":"😕",
    u"=L":"😕",
    u":S":"😕",
    u":‑\|":"😐",
    u":\|":"😐",
    u":$":"😳",
    u":‑x":"🤐",
    u":x":"🤐",
    u":‑#":"🤐",
    u":#":"🤐",
    u":‑&":"🤐",
    u":&":"🤐",
    u"O:‑\)":"😇",
    u"O:\)":"😇",
    u"0:‑3":"😇",
    u"0:3":"😇",
    u"0:‑\)":"😇",
    u"0:\)":"😇",
    u":‑b":"😛",
    u"0;\^\)":"😇",
    u">:‑\)":"😈",
    u">:\)":"😈",
    u"\}:‑\)":"😈",
    u"\}:\)":"😈",
    u"3:‑\)":"😈",
    u"3:\)":"😈",
    u">;\)":"😈",
    u"\|;‑\)":"😎",
    u"\|‑O":"😏",
    u":‑J":"😏",
    u"%‑\)":"😵",
    u"%\)":"😵",
    u":-###..":"🤒",
    u":###..":"🤒",
    u"\(>_<\)":"😣",
    u"\(>_<\)>":"😣",
    u"\(';'\)":"👶",
    u"\(\^\^>``":"😓",
    u"\(\^_\^;\)":"😓",
    u"\(-_-;\)":"😓",
    u"\(~_~;\) \(・\.・;\)":"😓",
    u"\(-_-\)zzz":"😴",
    u"\(\^_-\)":"😉",
    u"\(\(\+_\+\)\)":"😕",
    u"\(\+o\+\)":"😕",
    u"\^_\^":"😃",
    u"\(\^_\^\)/":"😃",
    u"\(\^O\^\)／":"😃",
    u"\(\^o\^\)／":"😃",
    u"\(__\)":"🙇",
    u"_\(\._\.\)_":"🙇",
    u"<\(_ _\)>":"🙇",
    u"<m\(__\)m>":"🙇",
    u"m\(__\)m":"🙇",
    u"m\(_ _\)m":"🙇",
    u"\('_'\)":"😭",
    u"\(/_;\)":"😭",
    u"\(T_T\) \(;_;\)":"😭",
    u"\(;_;":"😭",
    u"\(;_:\)":"😭",
    u"\(;O;\)":"😭",
    u"\(:_;\)":"😭",
    u"\(ToT\)":"😭",
    u";_;":"😭",
    u";-;":"😭",
    u";n;":"😭",
    u";;":"😭",
    u"Q\.Q":"😭",
    u"T\.T":"😭",
    u"QQ":"😭",
    u"Q_Q":"😭",
    u"\(-\.-\)":"😞",
    u"\(-_-\)":"😞",
    u"\(一一\)":"😞",
    u"\(；一_一\)":"😞",
    u"\(=_=\)":"😩",
    u"\(=\^\·\^=\)":"😺",
    u"\(=\^\·\·\^=\)":"😺",
    u"=_\^=	":"😺",
    u"\(\.\.\)":"😔",
    u"\(\._\.\)":"😔",
    u"\(\・\・?":"😕",
    u"\(?_?\)":"😕",
    u">\^_\^<":"😃",
    u"<\^!\^>":"😃",
    u"\^/\^":"😃",
    u"\（\*\^_\^\*）" :"😃",
    u"\(\^<\^\) \(\^\.\^\)":"😃",
    u"\(^\^\)":"😃",
    u"\(\^\.\^\)":"😃",
    u"\(\^_\^\.\)":"😃",
    u"\(\^_\^\)":"😃",
    u"\(\^\^\)":"😃",
    u"\(\^J\^\)":"😃",
    u"\(\*\^\.\^\*\)":"😃",
    u"\(\^—\^\）":"😃",
    u"\(#\^\.\^#\)":"😃",
    u"\（\^—\^\）":"👋",
    u"\(;_;\)/~~~":"👋",
    u"\(\^\.\^\)/~~~":"👋",
    u"\(T_T\)/~~~":"👋",
    u"\(ToT\)/~~~":"👋",
    u"\(\*\^0\^\*\)":"😍",
    u"\(\*_\*\)":"😍",
    u"\(\*_\*;":"😍",
    u"\(\+_\+\) \(@_@\)":"😍",
    u"\(\*\^\^\)v":"😂",
    u"\(\^_\^\)v":"😂",
    u'\(-"-\)':"😓",
    u"\(ーー;\)":"😓",
    u"\(\^0_0\^\)":"😎",
    u"\(\＾ｖ\＾\)":"😀",
    u"\(\＾ｕ\＾\)":"😀",
    u"\(\^\)o\(\^\)":"😀",
    u"\(\^O\^\)":"😀",
    u"\(\^o\^\)":"😀",
    u"\)\^o\^\(":"😀",
    u":O o_O":"😮",
    u"o_0":"😮",
    u"o\.O":"😮",
    u"\(o\.o\)":"😮",
    u"oO":"😮",
    u"\(\*￣m￣\)":"😠",
    u":‑)":"☺️",
    u":)":"☺️",
    u":-]":"☺️",
    u":]":"☺️",
    u":-3":"☺️",
    u":3":"☺️",
    u":->":"☺️",
    u":>":"☺️",
    u"8-)":"☺️",
    u":o)":"☺️",
    u":-}":"☺️",
    u":}":"☺️",
    u":-)":"☺️",
    u":c)":"☺️",
    u":^)":"☺️",
    u"=]":"☺️",
    u"=)":"☺️",
    u":‑D":"😃",
    u":D":"😃",
    u"8‑D":"😃",
    u"8D":"😃",
    u"X‑D":"😃",
    u"XD":"😃",
    u"=D":"😃",
    u"=3":"😃",
    u"B^D":"😃",
    u":-))":"😃",
    u":-(":"☹️",
    u":‑(":"☹️",
    u":(":"☹️",
    u":‑c":"☹️",
    u":c":"☹️",
    u":‑<":"☹️",
    u":<":"☹️",
    u":‑[":"☹️",
    u":[":"☹️",
    u":-||":"☹️",
    u">:[":"☹️",
    u":{":"☹️",
    u":@":"☹️",
    u">:(":"☹️",
    u":'‑(":"😭",
    u":'(":"😭",
    u":'‑)":"😃",
    u":')":"😃",
    u"D‑':":"😧",
    u"D:<":"😨",
    u"D:":"😧",
    u"D8":"😧",
    u"D;":"😧",
    u"D=":"😧",
    u"DX":"😧",
    u":‑O":"😮",
    u":O":"😮",
    u":‑o":"😮",
    u":o":"😮",
    u":-0":"😮",
    u"8‑0":"😮",
    u">:O":"😮",
    u":-*":"😗",
    u":*":"😗",
    u":X":"😗",
    u";‑)":"😉",
    u";)":"😉",
    u"*-)":"😉",
    u"*)":"😉",
    u";‑]":"😉",
    u";]":"😉",
    u";^)":"😉",
    u":‑,":"😉",
    u";D":"😉",
    u":‑P":"😛",
    u":P":"😛",
    u"X‑P":"😛",
    u"XP":"😛",
    u":‑Þ":"😛",
    u":Þ":"😛",
    u":b":"😛",
    u"d:":"😛",
    u"=p":"😛",
    u">:P":"😛",
    u":‑/":"😕",
    u":/":"😕",
    u":-[.]":"😕",
    u">:[(\)]":"😕",
    u">:/":"😕",
    u":[(\)]":"😕",
    u"=/":"😕",
    u"=[(\)]":"😕",
    u":L":"😕",
    u"=L":"😕",
    u":S":"😕",
    u":‑|":"😐",
    u":|":"😐",
    u":$":"😳",
    u":‑x":"🤐",
    u":x":"🤐",
    u":‑#":"🤐",
    u":#":"🤐",
    u":‑&":"🤐",
    u":&":"🤐",
    u"O:‑)":"😇",
    u"O:)":"😇",
    u"0:‑3":"😇",
    u"0:3":"😇",
    u"0:‑)":"😇",
    u"0:)":"😇",
    u":‑b":"😛",
    u"0;^)":"😇",
    u">:‑)":"😈",
    u">:)":"😈",
    u"}:‑)":"😈",
    u"}:)":"😈",
    u"3:‑)":"😈",
    u"3:)":"😈",
    u">;)":"😈",
    u"|;‑)":"😎",
    u"|‑O":"😏",
    u":‑J":"😏",
    u"%‑)":"😵",
    u"%)":"😵",
    u":-###..":"🤒",
    u":###..":"🤒",
    u"(>_<)":"😣",
    u"(>_<)>":"😣",
    u"(';')":"Baby",
    u"(^^>``":"😓",
    u"(^_^;)":"😓",
    u"(-_-;)":"😓",
    u"(~_~;) (・.・;)":"😓",
    u"(-_-)zzz":"😴",
    u"(^_-)":"😉",
    u"((+_+))":"😕",
    u"(+o+)":"😕",
    u"^_^":"😃",
    u"(^_^)/":"😃",
    u"(^O^)／":"😃",
    u"(^o^)／":"😃",
    u"(__)":"🙇",
    u"_(._.)_":"🙇",
    u"<(_ _)>":"🙇",
    u"<m(__)m>":"🙇",
    u"m(__)m":"🙇",
    u"m(_ _)m":"🙇",
    u"('_')":"😭",
    u"(/_;)":"😭",
    u"(T_T) (;_;)":"😭",
    u"(;_;":"😭",
    u"(;_:)":"😭",
    u"(;O;)":"😭",
    u"(:_;)":"😭",
    u"(ToT)":"😭",
    u";_;":"😭",
    u";-;":"😭",
    u";n;":"😭",
    u";;":"😭",
    u"Q.Q":"😭",
    u"T.T":"😭",
    u"QQ":"😭",
    u"Q_Q":"😭",
    u"(-.-)":"😞",
    u"(-_-)":"😞",
    u"(一一)":"😞",
    u"(；一_一)":"😞",
    u"(=_=)":"😩",
    u"(=^·^=)":"😺",
    u"(=^··^=)":"😺",
    u"=_^= ":"😺",
    u"(..)":"😔",
    u"(._.)":"😔",
    u"(・・?":"😕",
    u"(?_?)":"😕",
    u">^_^<":"😃",
    u"<^!^>":"😃",
    u"^/^":"😃",
    u"（*^_^*）" :"😃",
    u"(^<^) (^.^)":"😃",
    u"(^^)":"😃",
    u"(^.^)":"😃",
    u"(^_^.)":"😃",
    u"(^_^)":"😃",
    u"(^^)":"😃",
    u"(^J^)":"😃",
    u"(*^.^*)":"😃",
    u"(^—^）":"😃",
    u"(#^.^#)":"😃",
    u"（^—^）":"👋",
    u"(;_;)/~~~":"👋",
    u"(^.^)/~~~":"👋",
    u"(-_-)/~~~ ($··)/~~~":"👋",
    u"(T_T)/~~~":"👋",
    u"(ToT)/~~~":"👋",
    u"(*^0^*)":"😍",
    u"(*_*)":"😍",
    u"(*_*;":"😍",
    u"(+_+) (@_@)":"😍",
    u"(*^^)v":"😂",
    u"(^_^)v":"😂",
    u'(-"-)':"😓",
    u"(ーー;)":"😓",
    u"(^0_0^)":"😎",
    u"(＾ｖ＾)":"😀",
    u"(＾ｕ＾)":"😀",
    u"(^)o(^)":"😀",
    u"(^O^)":"😀",
    u"(^o^)":"😀",
    u")^o^(":"😀",
    u":O o_O":"😮",
    u"o_0":"😮",
    u"o.O":"😮",
    u"(o.o)":"😮",
    u"oO":"😮",
}


def str2emoji(tweet):
	
	for pos,ej in enumerate(tweet):
		if ej in emojis:
			tweet[pos]=emojis[ej]
	return tweet

In [5]:
def norm_tweet(tweet):
  tweet = re.sub(r"\\u2019", "'", tweet)
  tweet = re.sub(r"\\u002c", ",", tweet)
  tweet=' '.join(str2emoji(unidecode(tweet).lower().split()))
  tweet = re.sub(r"\'ve", " have", tweet)
  tweet = re.sub(r" can\'t", " cannot", tweet)
  tweet = re.sub(r"n\'t", " not", tweet)
  tweet = re.sub(r"\'re", " are", tweet)
  tweet = re.sub(r"\'d", " would", tweet)
  tweet = re.sub(r"\'ll", " will", tweet)
  tweet = re.sub(r"\'s", "", tweet)
  tweet = re.sub(r"\'n", "", tweet)
  tweet = re.sub(r"\'m", " am", tweet)
  tweet = re.sub(r"@\w+", r' ',tweet)
  tweet = re.sub(r"#\w+", r' ',tweet)
  tweet = re.sub(r"[.]+"," ",tweet)
  #T = tokenizer.TweetTokenizer()
  tweet = T.tokenize(tweet)
  #tweet = [lemmatizer.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v']  else lemmatizer.lemmatize(i) for i,j in pos_tag(tknzr.tokenize(tweet))]
  tweet = [ i for i in tweet if (i not in stopwords) and (i not in string.punctuation ) ]
  #tweet = ' '.join(tweet)
  return tweet

In [6]:
def data_preprocessing(data):
  data['data'] = data['data'].apply(norm_tweet)
  print(data['sentiment'].dtypes)
  if data['sentiment'].dtypes not in [np.int64]:
    data['sentiment']=data['sentiment'].apply(value_sentiment)
  return data['data'],data['sentiment'],data

In [7]:
def model1(x_train_3, y_train_3,x_val_3, y_val_3, embedding_layer):

	model1 = Sequential()
	model1.add(embedding_layer)
	model1.add(LSTM(32))
	model1.add(Dropout(0.2))
	model1.add(Dense(32, activation='relu'))
	model1.add(Dropout(0.2))
	model1.add(Dense(3, activation='softmax'))
	model1.compile(loss='categorical_crossentropy',
			      optimizer='Adam',
			      metrics=['acc'])
	model1.summary()
	history=model1.fit(x_train_3, y_train_3, validation_data=(x_val_3, y_val_3),epochs=6, batch_size=50)
	model1.save("./model1.h5")

In [8]:
def model2(x_train_3, y_train_3,x_val_3, y_val_3, embedding_layer,epochs, batch_size):
	model2 = Sequential()
	model2.add(embedding_layer)
	model2.add(GRU(32))
	model2.add(Dropout(0.2))
	model2.add(Dense(3, activation='softmax'))
	model2.compile(loss='categorical_crossentropy',
			      optimizer='rmsprop',
			      metrics=['acc'])
	model2.summary()
	history=model2.fit(x_train_3, y_train_3, validation_data=(x_val_3, y_val_3),epochs=epochs, batch_size=batch_size)
	model2.save("./model2.h5")

In [9]:
#Path of datas
data_train_3 =  "train_1_1.csv"
data_train_7 =  "train_3_3.csv"

In [10]:
#Load Data
data_train_3= pd.read_csv(data_train_3,sep='\t',names=['id','sentiment','data'])
data_train_7= pd.read_csv(data_train_7,sep='\t',names=['id','sentiment','data'])

In [11]:
# Apply the preprocessiog on the data
# Normalize the tweets
tweets_train_3, sentiments_train_3,data_train_3 = data_preprocessing(data_train_3)
tweets_train_7, sentiments_train_7,data_train_7 = data_preprocessing(data_train_7)

object
int64


In [12]:
all_tweet = tweets_train_3.append(tweets_train_7)
tokenizer = Tokenizer(filters=' ')
tokenizer.fit_on_texts(all_tweet)
word_index = tokenizer.word_index

In [13]:
sequences_train_3 = tokenizer.texts_to_sequences(tweets_train_3)
sequences_train_7 = tokenizer.texts_to_sequences(tweets_train_7)
sequences = sequences_train_3 + sequences_train_7


In [14]:
#Seek the longuest Tweet
#To Apply a Padding
MAX_SEQUENCE_LENGTH = 0
for elt in sequences:
	if len(elt) > MAX_SEQUENCE_LENGTH:
		MAX_SEQUENCE_LENGTH = len(elt)
  
data_train_3 = pad_sequences(sequences_train_3, maxlen=MAX_SEQUENCE_LENGTH)
data_train_7 = pad_sequences(sequences_train_7, maxlen=MAX_SEQUENCE_LENGTH)

In [15]:
indices_train_3 = np.arange(data_train_3.shape[0])
data_train_3 = data_train_3[indices_train_3]

indices_train_7 = np.arange(data_train_7.shape[0])
data_train_7 = data_train_7[indices_train_7]

labels_train_3 = to_categorical(np.asarray(sentiments_train_3), 3)
labels_train_3 = labels_train_3[indices_train_3]

In [16]:
nb_words=len(word_index)+1
EMBEDDING_DIM=300
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [17]:
oov=[]
oov.append((np.random.rand(EMBEDDING_DIM)) - 1.0)
oov = oov / np.linalg.norm(oov)

In [18]:
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
    else:
        embedding_matrix[i] = oov

In [19]:
split_idx = int(len(data_train_3)*0.70)
x_train_3, x_val_3 = data_train_3[:split_idx], data_train_3[split_idx:]
y_train_3, y_val_3 = labels_train_3 [:split_idx], labels_train_3[split_idx:]

In [20]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False, name='embedding_layer')

In [21]:
model1(x_train_3, y_train_3,x_val_3, y_val_3, embedding_layer)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 31, 300)           18438300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 18,482,079
Trainable params: 43,779
Non-trainable params: 18,438,300
_____________________________________

In [22]:
model2(x_train_3, y_train_3,x_val_3, y_val_3, embedding_layer, 6, 50)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 31, 300)           18438300  
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                31968     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 18,470,367
Trainable params: 32,067
Non-trainable params: 18,438,300
_________________________________________________________________
Train on 35233 samples, validate on 15100 samples
Epoch 1/6
35233/35233 [==============================] - 10s 280us/step - loss: 0.8374 - acc: 0.6112 - val_loss: 0.8103 - val_acc: 0.6108
Epoch 2/6
35233/35233 [========================

In [23]:
print(sentiments_train_7, tweets_train_7)
labels_train_7 = to_categorical(np.asarray(sentiments_train_7), 7)
labels_train_7 = labels_train_7[indices_train_7]

split_idx = int(len(data_train_7)*0.85)
x_train_7, x_val_7 = data_train_7[:split_idx], data_train_7[split_idx:]
y_train_7, y_val_7 = labels_train_7 [:split_idx], labels_train_7[split_idx:]

0       0
1       0
2       0
3       0
4      -2
       ..
1625    3
1626   -2
1627    1
1628    0
1629   -2
Name: sentiment, Length: 1630, dtype: int64 0                             [yeah, ☺, ️, playing, well]
1       [least, guy, trying, discourage, anymore, want...
2       [uplift, still, discouraged, means, listening,...
3                              [age, heyday, blood, tame]
4       [embarrassed, saw, us, like, knvfkkjg, thinks,...
                              ...                        
1625       [idk, help, someone, smile, comes, lips, n, n]
1626    [think, leep, favorite, get, dark, maybe, inso...
1627                   [amelia, want, sarah, v, grateful]
1628                                  [lack, makes, n, n]
1629                   [james, clapper, cary, disturbing]
Name: data, Length: 1630, dtype: object


In [27]:
model=keras.models.load_model("./model1.h5")
model.summary()
model.layers.pop()
model.layers.pop()
model.add(Dense(150,activation='relu',name='dense1'))
model.add(Dense(64,activation='relu',name='dense2'))
model.add(Dense(7,activation='softmax',name='dense3'))
model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])
history = model.fit(x_train_7, y_train_7,   validation_data=(x_val_7,y_val_7), epochs=11, batch_size=50)
model.save("./model3.h5")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 31, 300)           18438300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 18,482,079
Trainable params: 43,779
Non-trainable params: 18,438,300
_____________________________________

/home/beuvry_j/.local/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 1385 samples, validate on 245 samples
Epoch 1/11
1385/1385 [==============================] - 1s 575us/step - loss: 1.8115 - accuracy: 0.3040 - val_loss: 1.7134 - val_accuracy: 0.2735
Epoch 2/11
1385/1385 [==============================] - 0s 315us/step - loss: 1.6584 - accuracy: 0.3437 - val_loss: 1.6497 - val_accuracy: 0.2816
Epoch 3/11
1385/1385 [==============================] - 0s 319us/step - loss: 1.6028 - accuracy: 0.3545 - val_loss: 1.6366 - val_accuracy: 0.2939
Epoch 4/11
1385/1385 [==============================] - 0s 315us/step - loss: 1.5597 - accuracy: 0.3697 - val_loss: 1.6209 - val_accuracy: 0.3755
Epoch 5/11
1385/1385 [==============================] - 0s 316us/step - loss: 1.5282 - accuracy: 0.3776 - val_loss: 1.6350 - val_accuracy: 0.3510
Epoch 6/11
1385/1385 [==============================] - 0s 308us/step - loss: 1.4929 - accuracy: 0.4058 - val_loss: 1.6261 - val_accuracy: 0.3592
Epoch 7/11
1385/1385 [==============================] - 0s 317us/step - loss:

In [32]:
corpora_test_7="./train_3_3.csv"
corpora_test_7= pd.read_csv(corpora_test_7,sep='\t',names=['id','sentiment','data'])

tweets_test_7,sentiments_test_3,corpora_test_7 =  data_preprocessing(corpora_test_7)
sequences_test_7 = tokenizer.texts_to_sequences(tweets_test_7)

int64
